In [1]:
import numpy as np
import pandas as pd
import os
import re

In [2]:
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
print(os.getcwd())

/Users/sunnyyou/Documents/Real_Time_HAI


In [3]:
data = pd.read_csv('BRTK_SHIPS_2000to2019_IMERG_OK_Request_2023_FINAL.csv')
data['ID'] = data.GIS_ID.str[:10]
data = data[["GIS_ID", "JULDAY","LAND_OCEAN","ID"]]
data

,GIS_ID,JULDAY,LAND_OCEAN,ID
0,ATL_200001_TD_2000060718,2451703.25,Ocean,ATL_200001
1,ATL_200001_TD_2000060800,2451703.50,Ocean,ATL_200001
2,ATL_200001_TD_2000060806,2451703.75,Ocean,ATL_200001
3,ATL_200001_TD_2000060812,2451704.00,Ocean,ATL_200001
4,ATL_200002_TD_2000062300,2451718.50,Ocean,ATL_200002
...,...,...,...,...
40973,SPA_201927_TD_2019051506,2458618.75,Land,SPA_201927
40974,SPA_201927_TD_2019051512,2458619.00,Ocean,SPA_201927
40975,SPA_201927_TD_2019051518,2458619.25,Ocean,SPA_201927
40976,SPA_201927_TD_2019051600,2458619.50,Ocean,SPA_201927


In [4]:
s = data[data.ID == "ATL_200001"]
s

,GIS_ID,JULDAY,LAND_OCEAN,ID
0,ATL_200001_TD_2000060718,2451703.25,Ocean,ATL_200001
1,ATL_200001_TD_2000060800,2451703.50,Ocean,ATL_200001
2,ATL_200001_TD_2000060806,2451703.75,Ocean,ATL_200001
3,ATL_200001_TD_2000060812,2451704.00,Ocean,ATL_200001


In [5]:
final_df = pd.DataFrame(columns = ["GIS_ID", "LOP06", "LOP12", "LOP18", "LOP24"])

i=0
for i in range(data.shape[0]):
    if i%1000 == 0:
        print(i)
    row = data.loc[i]
    cur_time = row.JULDAY
    temp = data[data.ID == row.ID]
    p6_time = cur_time + 0.25
    p12_time = cur_time + 0.5
    p18_time = cur_time + 0.75
    p24_time = cur_time + 1.0

    p6 = temp[temp.JULDAY == p6_time]
    p12 = temp[temp.JULDAY == p12_time]
    p18 = temp[temp.JULDAY == p18_time]
    p24 = temp[temp.JULDAY == p24_time]

    if p6.shape[0] == 0:
        p6 = "-999"
    else: 
        p6 = p6.LAND_OCEAN.iloc[0]

    if p12.shape[0] == 0:
        p12 = "-999"
    else: 
        p12 = p12.LAND_OCEAN.iloc[0]

    if p18.shape[0] == 0:
        p18 = "-999"
    else: 
        p18 = p18.LAND_OCEAN.iloc[0]

    if p24.shape[0] == 0:
        p24 = "-999"
    else: 
        p24 = p24.LAND_OCEAN.iloc[0]

    final_df.loc[i] = [row.GIS_ID, p6, p12, p18, p24]

final_df


0


In [74]:
final_df = final_df[["GIS_ID", "LOP06", "LOP12", "LOP18", "LOP24"]]
final_df

,GIS_ID,LOP06,LOP12,LOP18,LOP24
0,ATL_200001_TD_2000060718,Ocean,Ocean,Ocean,-999
1,ATL_200001_TD_2000060800,Ocean,Ocean,-999,-999
2,ATL_200001_TD_2000060806,Ocean,-999,-999,-999
3,ATL_200001_TD_2000060812,-999,-999,-999,-999
4,ATL_200002_TD_2000062300,Ocean,Ocean,Ocean,Ocean
...,...,...,...,...,...
40973,SPA_201927_TD_2019051506,Ocean,Ocean,Ocean,-999
40974,SPA_201927_TD_2019051512,Ocean,Ocean,-999,-999
40975,SPA_201927_TD_2019051518,Ocean,-999,-999,Ocean
40976,SPA_201927_TD_2019051600,-999,-999,Ocean,-999


In [90]:
f = final_df
f["LOP12"] = np.where(f.LOP06 != "Ocean", f.LOP06, f.LOP12)
f["LOP18"] = np.where(f.LOP12 != "Ocean", f.LOP12, f.LOP18)
f["LOP24"] = np.where(f.LOP18 != "Ocean", f.LOP18, f.LOP24)

In [91]:
f

,GIS_ID,LOP06,LOP12,LOP18,LOP24
0,ATL_200001_TD_2000060718,Ocean,Ocean,Ocean,-999
1,ATL_200001_TD_2000060800,Ocean,Ocean,-999,-999
2,ATL_200001_TD_2000060806,Ocean,-999,-999,-999
3,ATL_200001_TD_2000060812,-999,-999,-999,-999
4,ATL_200002_TD_2000062300,Ocean,Ocean,Ocean,Ocean
...,...,...,...,...,...
40973,SPA_201927_TD_2019051506,Ocean,Ocean,Ocean,-999
40974,SPA_201927_TD_2019051512,Ocean,Ocean,-999,-999
40975,SPA_201927_TD_2019051518,Ocean,-999,-999,-999
40976,SPA_201927_TD_2019051600,-999,-999,-999,-999


,GIS_ID,LOP06,LOP12,LOP18,LOP24


In [93]:
original_data = pd.read_csv("BRTK_SHIPS_2000to2019_IMERG_OK_AUG08.csv")
original_data

,GIS_ID,BASIN,DATE,JULDAY,LATITUDE,LONGITUDE,MSLP,STORM_NAME,STORM_ID,CAT,...,TADV_4,TADV_5,TADV_6,TADV_7,TADV_8,TADV_9,TADV_10,TADV_11,TADV_12,TADV_13
0,ATL_200001_TD_2000060718,ATL,2000060718,2451703.25,21.000000,-93.000000,1008,NOT_NAMED,200001,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,ATL_200001_TD_2000060800,ATL,2000060800,2451703.50,20.900000,-92.800003,1009,NOT_NAMED,200001,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,ATL_200001_TD_2000060806,ATL,2000060806,2451703.75,20.700001,-93.099998,1010,NOT_NAMED,200001,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,ATL_200001_TD_2000060812,ATL,2000060812,2451704.00,20.799999,-93.500000,1010,NOT_NAMED,200001,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,ATL_200002_TD_2000062300,ATL,2000062300,2451718.50,9.500000,-19.799999,1010,NOT_NAMED,200002,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40973,SPA_201927_TD_2019051506,SPA,2019051506,2458618.75,-12.600000,142.399994,1008,ANN,201927,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
40974,SPA_201927_TD_2019051512,SPA,2019051512,2458619.00,-12.200000,141.000000,1009,ANN,201927,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
40975,SPA_201927_TD_2019051518,SPA,2019051518,2458619.25,-11.800000,139.600006,1009,ANN,201927,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
40976,SPA_201927_TD_2019051600,SPA,2019051600,2458619.50,-11.200000,138.500000,1008,ANN,201927,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [94]:
original_data = original_data.merge(f, on = "GIS_ID")
original_data

,GIS_ID,BASIN,DATE,JULDAY,LATITUDE,LONGITUDE,MSLP,STORM_NAME,STORM_ID,CAT,...,TADV_8,TADV_9,TADV_10,TADV_11,TADV_12,TADV_13,LOP06,LOP12,LOP18,LOP24
0,ATL_200001_TD_2000060718,ATL,2000060718,2451703.25,21.000000,-93.000000,1008,NOT_NAMED,200001,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,Ocean,Ocean,Ocean,-999
1,ATL_200001_TD_2000060800,ATL,2000060800,2451703.50,20.900000,-92.800003,1009,NOT_NAMED,200001,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,Ocean,Ocean,-999,-999
2,ATL_200001_TD_2000060806,ATL,2000060806,2451703.75,20.700001,-93.099998,1010,NOT_NAMED,200001,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,Ocean,-999,-999,-999
3,ATL_200001_TD_2000060812,ATL,2000060812,2451704.00,20.799999,-93.500000,1010,NOT_NAMED,200001,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999,-999,-999
4,ATL_200002_TD_2000062300,ATL,2000062300,2451718.50,9.500000,-19.799999,1010,NOT_NAMED,200002,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,Ocean,Ocean,Ocean,Ocean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40973,SPA_201927_TD_2019051506,SPA,2019051506,2458618.75,-12.600000,142.399994,1008,ANN,201927,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,Ocean,Ocean,Ocean,-999
40974,SPA_201927_TD_2019051512,SPA,2019051512,2458619.00,-12.200000,141.000000,1009,ANN,201927,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,Ocean,Ocean,-999,-999
40975,SPA_201927_TD_2019051518,SPA,2019051518,2458619.25,-11.800000,139.600006,1009,ANN,201927,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,Ocean,-999,-999,-999
40976,SPA_201927_TD_2019051600,SPA,2019051600,2458619.50,-11.200000,138.500000,1008,ANN,201927,TD,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999,-999,-999,-999


In [95]:
original_data.to_csv("Land_Ocean_Futures.csv")

In [39]:
cur_time + 0.25

2451703.5

In [40]:
.2

0.2